<a href="https://colab.research.google.com/github/plaban1981/Huggingface_transformers_course/blob/main/Next_Word_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout,Activation,Embedding
from string import punctuation
#
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from tensorflow.keras.utils import to_categorical

https://towardsdatascience.com/exploring-the-next-word-predictor-5e22aeb85d8f

https://renom.jp/notebooks/tutorial/time_series/text_generation_using_pretrained_word_embedding_layer/notebook.html

In [2]:
from google.colab import files
files.upload()

Saving HSBC_Credit_Card_FAQ.csv to HSBC_Credit_Card_FAQ.csv


{'HSBC_Credit_Card_FAQ.csv': b'Question,Answer\r\nWhich HSBC Credit Cards can I apply for?,"HSBC Credit Cards have been designed to complement your lifestyle. We currently offer credit cards which extend a host of offers ranging from travel privileges to dining, shopping and more.\nPlease visit the credit cards sectioncredit cards section View credit cards on offer from HSBC on our website for details on the various credit cards we offer, product features, benefits and the eligibility criteria."\r\nWhat documentation do I need to provide when applying for an HSBC Credit Card?,"Please note that the original documents need to be originally sighted and verified by a bank official:\n\x95\tSelf-attested recent passport size photograph of the applicant\n\x95\tSelf-attested copy of a valid photo identity document (please refer to the list below for details)\n\x95\tSelf-attested copy of your residence address proof (please refer to the list below for details)\n\x95\tSelf-attested copy of the l

In [3]:
import pandas as pd
df = pd.read_csv("/content/HSBC_Credit_Card_FAQ.csv",encoding='cp1252')

In [4]:
df.head()

,Question,Answer
0,Which HSBC Credit Cards can I apply for?,HSBC Credit Cards have been designed to comple...
1,What documentation do I need to provide when a...,Please note that the original documents need t...
2,How do I apply for an HSBC Credit Card?,To apply for an HSBC Credit Card you can:\n•\t...
3,"I have applied for an HSBC Credit Card, how ca...",You will receive timely alerts on the credit c...
4,How long does it take to receive an HSBC Credi...,Once we have received your application form co...


In [ ]:
df.tail()

,Question,Answer
123,Can I decrease the credit limit on my Credit C...,"Yes, you may decrease the limit if your circum..."
124,Will I be intimated if my credit limit is redu...,"Yes, post the credit limit reduction you will ..."
125,Will my limit on my Add on card get increased/...,"Yes, if the limit on your primary credit card ..."
126,What happens if my credit limit request is dec...,If the credit request is rejected due to inter...
127,How do I contact HSBC for any queries or servi...,All account holders\nThe email address 'info@h...


In [5]:
text = df['Question'].values.tolist()

In [6]:
len(text)

128

In [35]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
words = []
for txt in text:
  words.extend(tokenizer.tokenize(txt.lower()))

In [36]:
words

['which',
 'hsbc',
 'credit',
 'cards',
 'can',
 'i',
 'apply',
 'for',
 'what',
 'documentation',
 'do',
 'i',
 'need',
 'to',
 'provide',
 'when',
 'applying',
 'for',
 'an',
 'hsbc',
 'credit',
 'card',
 'how',
 'do',
 'i',
 'apply',
 'for',
 'an',
 'hsbc',
 'credit',
 'card',
 'i',
 'have',
 'applied',
 'for',
 'an',
 'hsbc',
 'credit',
 'card',
 'how',
 'can',
 'i',
 'check',
 'on',
 'the',
 'status',
 'of',
 'this',
 'how',
 'long',
 'does',
 'it',
 'take',
 'to',
 'receive',
 'an',
 'hsbc',
 'credit',
 'card',
 'how',
 'can',
 'i',
 'check',
 'the',
 'status',
 'of',
 'my',
 'credit',
 'card',
 'consignment',
 'i',
 'have',
 'not',
 'received',
 'my',
 'card',
 'statement',
 'how',
 'do',
 'i',
 'register',
 'for',
 'e',
 'statements',
 'can',
 'i',
 'use',
 'my',
 'hsbc',
 'credit',
 'card',
 'as',
 'soon',
 'as',
 'i',
 'receive',
 'it',
 'how',
 'can',
 'i',
 'change',
 'my',
 'pin',
 'what',
 'fees',
 'and',
 'charges',
 'are',
 'linked',
 'to',
 'the',
 'credit',
 'card',
 

In [38]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))
#unique_word_index = dict((c, i) for i, c in enumerate(words))

In [39]:
WORD_LENGTH = 3
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['which', 'hsbc', 'credit']
cards


In [40]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [55]:
print(np.argmax(X[0][0]))

279


## Building the model

In [44]:
WORD_LENGTH

3

In [58]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

## Training

In [59]:
from keras.optimizers import RMSprop
optimizer = RMSprop(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=500, shuffle=True).history

Epoch 1/500
12/12 [==============================] - 2s 62ms/step - loss: 5.6575 - accuracy: 0.0482 - val_loss: 5.6125 - val_accuracy: 0.1139
Epoch 2/500
12/12 [==============================] - 0s 5ms/step - loss: 5.5451 - accuracy: 0.1151 - val_loss: 5.3883 - val_accuracy: 0.0506
Epoch 3/500
12/12 [==============================] - 0s 5ms/step - loss: 5.1508 - accuracy: 0.0741 - val_loss: 4.9589 - val_accuracy: 0.0506
Epoch 4/500
12/12 [==============================] - 0s 5ms/step - loss: 4.7036 - accuracy: 0.0630 - val_loss: 4.9595 - val_accuracy: 0.0506
Epoch 5/500
12/12 [==============================] - 0s 5ms/step - loss: 4.5657 - accuracy: 0.0749 - val_loss: 5.0056 - val_accuracy: 0.0506
Epoch 6/500
12/12 [==============================] - 0s 5ms/step - loss: 4.5031 - accuracy: 0.0911 - val_loss: 5.0467 - val_accuracy: 0.0506
Epoch 7/500
12/12 [==============================] - 0s 5ms/step - loss: 4.4952 - accuracy: 0.0833 - val_loss: 5.0902 - val_accuracy: 0.0886
Epoch 8/500


In [60]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("Which HSBC Credit ".lower())

which
hsbc
credit


array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [61]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [62]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    words = [unique_words[idx] for idx in next_indices]
    words = [ w for w in words if w not in text.split(" ")]
    return words

In [67]:
import pickle
import heapq
q =  input()
print("correct sentence: ",q.lower())
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 3))

can i
correct sentence:  can i
Sequence:  can i
can
i
next possible words:  ['the', 'balance', 'my']


## New Method

## Tokenize input



In [7]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
words = []
for txt in text:
  words.extend(tokenizer.tokenize(txt.lower()))

In [8]:
len(words)

1564

## Preprocessing the data

* we are keeping training length to 3 so that we type atleast 3 words and have the next word predicted
* Generate Text Sequences
* Create features dictionary sequences. 
* Then encode it into the integer form with the help of the Tokenizer.

In [72]:
tokens = words
train_len = 3+1 # it tells how many words we need to type
text_sequences = []
for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)
sequences = {}
count = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]


In [73]:
text_sequences

[['which', 'hsbc', 'credit', 'cards'],
 ['hsbc', 'credit', 'cards', 'can'],
 ['credit', 'cards', 'can', 'i'],
 ['cards', 'can', 'i', 'apply'],
 ['can', 'i', 'apply', 'for'],
 ['i', 'apply', 'for', 'what'],
 ['apply', 'for', 'what', 'documentation'],
 ['for', 'what', 'documentation', 'do'],
 ['what', 'documentation', 'do', 'i'],
 ['documentation', 'do', 'i', 'need'],
 ['do', 'i', 'need', 'to'],
 ['i', 'need', 'to', 'provide'],
 ['need', 'to', 'provide', 'when'],
 ['to', 'provide', 'when', 'applying'],
 ['provide', 'when', 'applying', 'for'],
 ['when', 'applying', 'for', 'an'],
 ['applying', 'for', 'an', 'hsbc'],
 ['for', 'an', 'hsbc', 'credit'],
 ['an', 'hsbc', 'credit', 'card'],
 ['hsbc', 'credit', 'card', 'how'],
 ['credit', 'card', 'how', 'do'],
 ['card', 'how', 'do', 'i'],
 ['how', 'do', 'i', 'apply'],
 ['do', 'i', 'apply', 'for'],
 ['i', 'apply', 'for', 'an'],
 ['apply', 'for', 'an', 'hsbc'],
 ['for', 'an', 'hsbc', 'credit'],
 ['an', 'hsbc', 'credit', 'card'],
 ['hsbc', 'credit', '

In [74]:
n_sequences.shape

(1560, 4)

In [75]:
text_sequences[0]

['which', 'hsbc', 'credit', 'cards']

In [13]:
n_sequences[0]

array([45, 13,  5, 57], dtype=int32)

* The numbers are nothing but the indexes of the respective words from the ‘sequences’ dictionary before re-assignment.

In [14]:
vocabulary_size

291

* Once we have our sequences in encoded form training data and target data is defined by splitting the sequences into the inputs and output labels. 

* As for this example, we are going to predict the next word based on three previous words so in training we use the first three words as input and the last word as a label that is to be predicted by the model. Our ‘training_inputs’ would now be:

In [76]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
train_inputs.shape

(1560, 3)

In [20]:
seq_len

3

In [29]:
train_inputs[0],np.argmax(train_targets[0])

(array([45, 13,  5], dtype=int32), 57)

In [30]:
train_targets[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [23]:
np.argmax(train_targets[0])

57

## Build Model

In [54]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,validation_split=0.10,epochs=600,verbose=1)
model.save("mymodel.h5")

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 3, 3)              873       
_________________________________________________________________
lstm_7 (LSTM)                (None, 3, 50)             10800     
_________________________________________________________________
lstm_8 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_8 (Dense)              (None, 291)               14841     
Total params: 49,264
Trainable params: 49,264
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/600
44/44 [==============================] - 4s 25ms/step - loss: 5.6568 - accuracy: 0.0549 - val_loss:

## Copy the saved model to google drive

In [15]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [32]:
import shutil
shutil.copy("/content/mymodel.h5","/content/drive/MyDrive/ZeonAI/")

'/content/drive/MyDrive/ZeonAI/mymodel.h5'

## Make Inference on new text for the next 3 possible words

In [78]:
from keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

 if my credit 
[16, 4, 5] [[16  4  5]]
Next word suggestion: limit
Next word suggestion: card
Next word suggestion: credit


In [68]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
#
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from tensorflow.keras.utils import to_categorical
#
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
#
tokenizer = Tokenizer()
seq_len =3
def next_word_model():
  model = Sequential()
  model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
  model.add(LSTM(50,return_sequences=True))
  model.add(LSTM(50))
  model.add(Dense(50,activation='relu'))
  model.add(Dense(vocabulary_size, activation='softmax'))
  print(model.summary())
  return model
  


In [71]:
tokenizer.texts_to_sequences(["how can i"])

[[]]

## Loading the saved model and making inference

In [80]:
from keras.preprocessing.sequence import pad_sequences
"""
model = next_word_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.load_weights("/content/mymodel.h5")
#model = load_model("/content/drive/MyDrive/ZeonAI/mymodel.h5")
"""
model = next_word_model()
model = load_model("/content/drive/MyDrive/ZeonAI/mymodel.h5")
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
print(encoded_text,seq_len)
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 3, 3)              873       
_________________________________________________________________
lstm_16 (LSTM)               (None, 3, 50)             10800     
_________________________________________________________________
lstm_17 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_16 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_17 (Dense)             (None, 291)               14841     
Total params: 49,264
Trainable params: 49,264
Non-trainable params: 0
_________________________________________________________________
None
HSBC for any 
[13, 11, 82] 3
[[13 11 82]]
Next word suggestion: queries
Next word suggestion: as
Next word sugg